Importing Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Importing the data set

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop(['Ticket'],axis=1,inplace=True)
train.drop(['Name'],axis=1,inplace=True)
train.drop(['Cabin'],axis=1,inplace=True)
train.drop(['PassengerId'],axis=1,inplace=True)

test.drop(['Ticket'],axis=1,inplace=True)
test.drop(['Name'],axis=1,inplace=True)
test.drop(['Cabin'],axis=1,inplace=True)
test.drop(['PassengerId'],axis=1,inplace=True)



In [172]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    417 non-null    int64  
 1   Sex       417 non-null    object 
 2   Age       417 non-null    float64
 3   SibSp     417 non-null    int64  
 4   Parch     417 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  417 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 26.1+ KB


In [140]:
train.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


Taking care of missing values

In [0]:
def impute_age(cols):
  Age = cols[0]
  Pclass = cols[1]

  if pd.isnull(Age):
    if Pclass == 1:
      return 37
    elif Pclass == 2:
      return 29
    else:
      return 24
  else:
    return Age

train['Age'] = train[['Age','Pclass']].apply(impute_age,axis= 1)
test['Age'] = test[['Age','Pclass']].apply(impute_age,axis= 1)

In [142]:
train.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [0]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [0]:
survived = list(train.iloc[:,0])
train.drop(['Survived'],axis=1,inplace=True)

train['Survived'] = survived
X_train = train.iloc[:, :-1].values
y_train = train.iloc[:, -1].values
X_test =  test.iloc[:,:].values
y_test = test.iloc[:,-2]

In [159]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,10
0,0,1,0,1,0,0,3,22,1,0,7.25
1,1,0,1,0,1,0,1,38,1,0,71.2833
2,0,1,1,0,0,0,3,26,0,0,7.925
3,0,1,1,0,0,0,1,35,1,0,53.1
4,0,1,0,1,0,0,3,35,0,0,8.05


Encoding the categorical variables

In [156]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.fit_transform(X_test))



pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8
0,0,0,1,3,male,22,1,0,7.25
1,1,0,0,1,female,38,1,0,71.2833
2,0,0,1,3,female,26,0,0,7.925
3,0,0,1,1,female,35,1,0,53.1
4,0,0,1,3,male,35,0,0,8.05


In [0]:
ct1 = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [4])], remainder='passthrough')
X_train = np.array(ct1.fit_transform(X_train))

X_test = np.array(ct1.fit_transform(X_test))

Feature Scaling

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Training the model

In [161]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

Predicting the dependent variable

In [0]:
y_pred = classifier.predict(X_test)
y_pred1 = classifier.predict(X_train)

Confusion Matrix

In [163]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_pred1)
print(cm)

[[521  28]
 [110 230]]


K-fold Cross Validation

In [164]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 82.12 %
Standard Deviation: 3.70 %


Saving the prediction to a csv file

In [0]:
df = pd.DataFrame(y_pred)
df.to_csv('my_submission.csv')